## CIFAR-10 데이터셋 다운로드

In [ ]:
from tensorflow.keras.datasets import cifar10
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 평균과 표준편차는 채널별로 구해줍니다.
norm_layer = tf.keras.layers.Normalization(axis = -1, dtype = 'float32')
norm_layer.adapt(x_train)

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, 
                                                  test_size = 0.3)
print(x_train.shape, len(y_train))
print(x_val.shape, len(y_val))

img_shape = x_train.shape[1:]

## TensorFlow Hub에서 모델 가져오기

In [ ]:
import tensorflow_hub as hub # !pip install --upgrade tensorflow_hub

# 참조: https://tfhub.dev/google/efficientnet/b0/classification/1
model_url = "https://tfhub.dev/google/efficientnet/b0/classification/1"
hub_layer = hub.KerasLayer(model_url, trainable = False, input_shape = img_shape)

## Hub_Layer를 활용한 학습

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input

model = Sequential([
    Input(shape = img_shape),
    norm_layer,
    hub_layer,
    Dense(10, activation = 'softmax')
])

model.compile(loss = 'sparse_categorical_crossentropy',
              metrics = ['acc'],
              optimizer = 'adam')

In [ ]:
model.summary()